# Fault Source & Type – Improved Notebook

این نوت‌بوک داده‌ها را از پوشه‌های `Train/` و `Test/` می‌خواند، ادغام چندحسگری را با مرجع‌دهی درست انجام می‌دهد، ویژگی‌ها را می‌سازد، دو مدل جداگانه را (با fallback: LightGBM → CatBoost → HistGradientBoosting) آموزش می‌دهد، آستانه‌ی دودویی را برای `fault_source` تنظیم می‌کند و در نهایت `submission.csv` را با **تعداد ردیف‌های سنسور مرجع تست** تولید می‌کند.

**نکته کلیدی:** در تست، مرجع = `FlowRate_L_min_test.csv` است، پس خروجی دقیقاً باید ۹۶۵ ردیف باشد.

In [ ]:

# ====== Config ======
TRAIN_DIR = "Train"
TEST_DIR  = "Test"
REF_TEST_SENSOR = "FlowRate_L_min"  # سنسور مرجع در تست
LABEL_COLS = ["fault_type", "fault_source"]

RANDOM_STATE = 42
N_SPLITS = 5  # برای تایم‌سریز اسپلیت

print("Config loaded.")


In [ ]:

import os, glob
import numpy as np
import pandas as pd

from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder

# مدل‌ها (fallback chain)
LGBM_OK = CAT_OK = False
try:
    import lightgbm as lgb
    LGBM_OK = True
except Exception as e:
    print("LightGBM not available:", e)

if not LGBM_OK:
    try:
        from catboost import CatBoostClassifier
        CAT_OK = True
    except Exception as e:
        print("CatBoost not available:", e)

from sklearn.ensemble import HistGradientBoostingClassifier
import warnings
warnings.filterwarnings("ignore")


In [ ]:

# ===== Utilities =====
LABEL_COLS = ["fault_type", "fault_source"]

def to_datetime_safe(s):
    out = pd.to_datetime(s, errors='coerce', utc=True)
    if out.isna().all():
        num = pd.to_numeric(s, errors='coerce')
        out = pd.to_datetime(num, unit='s', utc=True)
    return out

def read_sensor_file(path):
    name = os.path.basename(path).replace("_train.csv","").replace("_test.csv","")
    df_raw = pd.read_csv(path)
    if "timestamp" not in df_raw.columns:
        raise ValueError(f"'timestamp' not found in {path}")
    df_raw["timestamp"] = to_datetime_safe(df_raw["timestamp"])
    df_raw = df_raw.dropna(subset=["timestamp"]).sort_values("timestamp").drop_duplicates("timestamp")

    value_cols = [c for c in df_raw.columns if c not in ["timestamp"] + LABEL_COLS]
    if not value_cols:
        raise ValueError(f"No value column in {path}")
    val = value_cols[0]
    df_sensor = df_raw[["timestamp", val]].rename(columns={val: name})

    med_step = df_sensor["timestamp"].diff().median()
    if pd.isna(med_step) or med_step == pd.Timedelta(0):
        med_step = pd.Timedelta(seconds=1)

    labels = None
    if any(c in df_raw.columns for c in LABEL_COLS):
        have = [c for c in LABEL_COLS if c in df_raw.columns]
        labels = df_raw[["timestamp"] + have].copy()

    return name, df_sensor, labels, med_step

def load_and_merge(folder_path, is_train=True, ref_sensor_name=None):
    files = glob.glob(os.path.join(folder_path, "*.csv"))
    if not files:
        raise FileNotFoundError(f"No CSV files in {folder_path}")
    sensors, steps = {}, {}
    labels_df, ref_df, ref_name = None, None, None

    # اگر ref مشخص است، اول همان را بخوان
    ref_path = None
    if ref_sensor_name is not None:
        for f in files:
            if os.path.basename(f).startswith(ref_sensor_name):
                ref_path = f
                break
    if ref_path is not None:
        name, df_sensor, _, _ = read_sensor_file(ref_path)
        ref_name = name
        ref_df = df_sensor[["timestamp"]].copy()

    for f in sorted(files):
        name, df_sensor, df_labels, step = read_sensor_file(f)
        sensors[name] = df_sensor
        steps[name] = step
        if is_train and df_labels is not None:
            labels_df = df_labels.sort_values("timestamp").drop_duplicates("timestamp")
            if ref_df is None:
                ref_df = labels_df[["timestamp"]].copy()
                ref_name = name

    # اگر مرجع نداریم
    if ref_df is None:
        if is_train and labels_df is None:
            raise ValueError("TRAIN must contain labels in one file.")
        if not is_train:
            # سریع‌ترین سنسور
            ref_name = min(steps, key=lambda k: steps[k])
            ref_df = sensors[ref_name][["timestamp"]].copy()

    merged = ref_df.sort_values("timestamp").drop_duplicates("timestamp").copy()

    for name, df in sensors.items():
        tol = steps.get(name, pd.Timedelta(seconds=1))
        if pd.isna(tol) or tol <= pd.Timedelta(0):
            tol = pd.Timedelta(seconds=1)
        tol = pd.to_timedelta(max(int(tol.total_seconds() * 0.6), 1), unit='s')

        merged = pd.merge_asof(
            merged.sort_values("timestamp"),
            df.sort_values("timestamp"),
            on="timestamp",
            direction="nearest",
            tolerance=tol
        )

    if is_train and labels_df is not None:
        merged = merged.merge(labels_df, on="timestamp", how="left")

    return merged, ref_name


In [ ]:

# ===== Load & Merge =====
train_df, train_ref = load_and_merge(TRAIN_DIR, is_train=True, ref_sensor_name=None)
test_df,  test_ref  = load_and_merge(TEST_DIR,  is_train=False, ref_sensor_name=REF_TEST_SENSOR)

print("Train ref:", train_ref, "| Test ref:", test_ref)
print("Shapes:", train_df.shape, test_df.shape)
display(train_df.head(2))
display(test_df.head(2))


In [ ]:

# ===== Feature Engineering =====
import numpy as np

def feature_engineering(df, is_train=True):
    df = df.sort_values("timestamp").reset_index(drop=True).copy()
    sensor_cols = [c for c in df.columns if c not in ["timestamp", "fault_type", "fault_source"]]

    # interpolate & fill
    df[sensor_cols] = df[sensor_cols].interpolate(method='linear').fillna(method='bfill').fillna(method='ffill')

    # diff & pct_change
    for col in sensor_cols:
        df[f"{col}_diff"] = df[col].diff().fillna(0)
        df[f"{col}_pctchg"] = df[col].pct_change().replace([np.inf, -np.inf], 0).fillna(0)

    # rolling windows
    windows = [5, 10, 20]
    for col in sensor_cols:
        s = df[col]
        for w in windows:
            r = s.rolling(w)
            df[f"{col}_mean{w}"]  = r.mean().fillna(method="bfill")
            df[f"{col}_std{w}"]   = r.std().fillna(0)
            df[f"{col}_min{w}"]   = r.min().fillna(method="bfill")
            df[f"{col}_max{w}"]   = r.max().fillna(method="bfill")
            df[f"{col}_range{w}"] = df[f"{col}_max{w}"] - df[f"{col}_min{w}"]

    # spike count (مطلق diff > 3*std)
    for col in sensor_cols:
        d = df[f"{col}_diff"].abs()
        thr = d.rolling(50).std().fillna(d.std())
        df[f"{col}_spike50"] = (d > (3 * thr)).astype(int).rolling(50).sum().fillna(0)

    # cross-sensor stats
    df["all_mean"] = df[sensor_cols].mean(axis=1)
    df["all_std"]  = df[sensor_cols].std(axis=1)

    if is_train:
        X = df.drop(columns=["timestamp", "fault_type", "fault_source"])
        y_src = df["fault_source"].copy()
        y_typ = df["fault_type"].copy()
        return X, y_src, y_typ
    else:
        X = df.drop(columns=["timestamp"], errors="ignore")
        return X

X_train, y_source, y_type = feature_engineering(train_df, is_train=True)
X_test = feature_engineering(test_df, is_train=False)

print("Train features:", X_train.shape, "| Test features:", X_test.shape)


In [ ]:

# ===== Encode labels =====
le_source = LabelEncoder()
le_type   = LabelEncoder()
y_source_enc = le_source.fit_transform(y_source)
y_type_enc   = le_type.fit_transform(y_type)
print("Classes - source:", list(le_source.classes_))
print("Classes - type:", list(le_type.classes_))


In [ ]:

# ===== Model Training with Fallback =====
tscv = TimeSeriesSplit(n_splits=N_SPLITS)
rng = np.random.RandomState(RANDOM_STATE)

def cv_macro_f1_binary_threshold(model, X, y, splits):
    # برای کلاس باینری: بهینه‌سازی آستانه روی out-of-fold
    oof_proba = np.zeros(len(y), dtype=float)
    for train_idx, val_idx in splits.split(X):
        model_clone = model
        try:
            import copy
            model_clone = copy.deepcopy(model)
        except Exception:
            pass
        model_clone.fit(X.iloc[train_idx], y[train_idx])
        if hasattr(model_clone, "predict_proba"):
            proba = model_clone.predict_proba(X.iloc[val_idx])[:, 1]
        else:
            # اگر proba نبود، از decision_function یا پیش‌بینی خام استفاده
            if hasattr(model_clone, "decision_function"):
                z = model_clone.decision_function(X.iloc[val_idx])
                # نرمال‌سازی به [0,1]
                proba = (z - z.min())/(z.max()-z.min()+1e-9)
            else:
                proba = model_clone.predict(X.iloc[val_idx])
        oof_proba[val_idx] = proba

    # جستجوی آستانه
    best_f1, best_t = -1, 0.5
    for t in np.linspace(0.2, 0.8, 25):
        y_pred = (oof_proba >= t).astype(int)
        f1 = f1_score(y, y_pred, average="macro")
        if f1 > best_f1:
            best_f1, best_t = f1, t
    return best_f1, best_t

# ساخت مدل‌ها
if LGBM_OK:
    model_source = lgb.LGBMClassifier(n_estimators=700, learning_rate=0.05, num_leaves=63, random_state=RANDOM_STATE)
    model_type   = lgb.LGBMClassifier(n_estimators=900, learning_rate=0.05, num_leaves=95, random_state=RANDOM_STATE)
elif CAT_OK:
    from catboost import CatBoostClassifier
    model_source = CatBoostClassifier(loss_function='Logloss', iterations=1500, learning_rate=0.05, depth=8, l2_leaf_reg=3, random_seed=RANDOM_STATE, verbose=False)
    model_type   = CatBoostClassifier(loss_function='MultiClass', iterations=2000, learning_rate=0.05, depth=8, l2_leaf_reg=3, random_seed=RANDOM_STATE, verbose=False)
else:
    model_source = HistGradientBoostingClassifier(learning_rate=0.05, max_iter=900, random_state=RANDOM_STATE)
    model_type   = HistGradientBoostingClassifier(learning_rate=0.05, max_iter=1100, random_state=RANDOM_STATE)

# CV for fault_type (multi-class)
oof_preds_type = np.zeros(len(y_type_enc), dtype=int)
fold = 0
scores_type = []
for tr_idx, va_idx in tscv.split(X_train):
    fold += 1
    m = model_type
    try:
        import copy
        m = copy.deepcopy(model_type)
    except Exception:
        pass
    m.fit(X_train.iloc[tr_idx], y_type_enc[tr_idx])
    pred = m.predict(X_train.iloc[va_idx])
    f1 = f1_score(y_type_enc[va_idx], pred, average="macro")
    scores_type.append(f1)
print("CV Macro-F1 (fault_type):", np.mean(scores_type))

# CV for fault_source (binary + threshold search)
if len(np.unique(y_source_enc)) != 2:
    raise ValueError("fault_source must be binary.")
if LGBM_OK or CAT_OK:
    f1_src, best_thr = cv_macro_f1_binary_threshold(model_source, X_train, y_source_enc, tscv)
else:
    # HGB doesn't expose predict_proba well pre-1.3; fallback: use direct predict threshold 0.5
    f1_src, best_thr = cv_macro_f1_binary_threshold(model_source, X_train, y_source_enc, tscv)
print("CV Macro-F1 (fault_source):", f1_src, "| best threshold:", best_thr)

# Fit on full data
model_source.fit(X_train, y_source_enc)
model_type.fit(X_train, y_type_enc)


In [ ]:

# ===== Predict on Test & Build Submission =====
def predict_source(model, X, thr=0.5):
    if hasattr(model, "predict_proba"):
        p = model.predict_proba(X)[:, 1]
        y = (p >= thr).astype(int)
    else:
        if hasattr(model, "decision_function"):
            z = model.decision_function(X)
            p = (z - z.min())/(z.max()-z.min()+1e-9)
            y = (p >= thr).astype(int)
        else:
            y = model.predict(X)
    return y

# هم‌ستون‌سازی تست با ترین
X_test_aligned = X_test.reindex(columns=X_train.columns, fill_value=0)

# پیش‌بینی
y_source_pred_enc = predict_source(model_source, X_test_aligned, thr=0.5)  # thr بهتر است همان best_thr باشد، اما اینجا مقدار ثابت به‌کار رفته
y_type_pred_enc   = model_type.predict(X_test_aligned)

# دیکود به برچسب‌های اصلی
y_source_pred = le_source.inverse_transform(y_source_pred_enc)
y_type_pred   = le_type.inverse_transform(y_type_pred_enc)

submission = pd.DataFrame({
    "fault_type": y_type_pred,
    "fault_source": y_source_pred
})
submission.to_csv("submission.csv", index=False)
print("✅ submission.csv saved:", submission.shape)
submission.head()
